In [ ]:
import numpy as np
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.building import add_meshcat_to_builder, finish_build
from cloth_tools.drake.scenes import add_dual_ur5e_and_table_to_builder
from cloth_tools.ompl.dual_arm_planner import DualArmOmplPlanner
from cloth_tools.drake.visualization import publish_dual_arm_joint_path, publish_ik_solutions, add_meshcat_triad

In [ ]:
tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175

# For table mounted setup: shoulder outside this range will almost certainly collide with the table
# For all UR robots: elbow constrainted -160 to 160 due to self-collision
joint_bounds_lower = np.deg2rad([-360, -180, -160, -360, -360, -360])
joint_bounds_upper = np.deg2rad([360, 0, 160, 360, 360, 360])
joint_bounds = (joint_bounds_lower, joint_bounds_upper)

In [ ]:
# Creating the default scene
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_indices, gripper_indices = add_dual_ur5e_and_table_to_builder(robot_diagram_builder)
diagram, context = finish_build(robot_diagram_builder, meshcat)

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[*arm_indices, *gripper_indices],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

is_state_valid_fn = collision_checker.CheckConfigCollisionFree

In [ ]:
home_joints_left = np.deg2rad([180, -120, 60, -30, -90, -90])
home_joints_right = np.deg2rad([-180, -60, -60, -150, 90, 90])

In [ ]:
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

arm_left_index, arm_right_index = arm_indices
plant.SetPositions(plant_context, arm_left_index, home_joints_left)
plant.SetPositions(plant_context, arm_right_index, home_joints_right)
diagram.ForcedPublish(context)

In [ ]:
transform_0 = RigidTransform(p=[0, 0, 0.35], rpy=RollPitchYaw([np.pi, 0, 0]))
tcp_pose_0 = np.ascontiguousarray(transform_0.GetAsMatrix4())

add_meshcat_triad(meshcat, "TCP Frame left", X_W_Triad=transform_0)

In [ ]:
transform_1 = RigidTransform(p=[0.15, 0, 0.3], rpy=RollPitchYaw([np.pi / 2, 0, np.pi / 2]))
tcp_pose_1 = np.ascontiguousarray(transform_1.GetAsMatrix4())

add_meshcat_triad(meshcat, "TCP Frame right", X_W_Triad=transform_1)

In [ ]:
from functools import partial
from typing import List
from airo_typing import HomogeneousMatrixType, JointConfigurationType
from ur_analytic_ik import ur5e

from cloth_tools.drake.scenes import X_W_L_DEFAULT, X_W_R_DEFAULT, X_CB_B

X_W_LCB = X_W_L_DEFAULT @ X_CB_B.inverse()
X_W_RCB = X_W_R_DEFAULT @ X_CB_B.inverse()


def inverse_kinematics_in_world_fn(
    tcp_pose: HomogeneousMatrixType, X_W_CB: HomogeneousMatrixType
) -> List[JointConfigurationType]:
    X_W_TCP = tcp_pose
    X_CB_W = np.linalg.inv(X_W_CB)
    solutions_1x6 = ur5e.inverse_kinematics_with_tcp(X_CB_W @ X_W_TCP, tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions


inverse_kinematics_left_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_LCB.GetAsMatrix4())
inverse_kinematics_right_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_RCB.GetAsMatrix4())

In [ ]:
solutions_left = inverse_kinematics_left_fn(tcp_pose_0)
publish_ik_solutions(solutions_left, 2.0, meshcat, diagram, context, arm_left_index)

In [ ]:
solutions_right = inverse_kinematics_right_fn(tcp_pose_1)
publish_ik_solutions(solutions_right, 2.0, meshcat, diagram, context, arm_right_index)

In [ ]:
planner = DualArmOmplPlanner(
    is_state_valid_fn,
    inverse_kinematics_left_fn,
    inverse_kinematics_right_fn,
    joint_bounds_left=joint_bounds,
    joint_bounds_right=joint_bounds
)

In [ ]:
path = planner.plan_to_tcp_pose(home_joints_left, home_joints_right, tcp_pose_0, None)
publish_dual_arm_joint_path(path, 2.0, meshcat, diagram, context, *arm_indices)

In [ ]:
path = planner.plan_to_tcp_pose(home_joints_left, home_joints_right, None, tcp_pose_1)
publish_dual_arm_joint_path(path, 2.0, meshcat, diagram, context, *arm_indices)

In [ ]:
path = planner.plan_to_tcp_pose(home_joints_left, home_joints_right, tcp_pose_0, tcp_pose_1)
publish_dual_arm_joint_path(path, 2.0, meshcat, diagram, context, *arm_indices)

In [ ]:
from cloth_tools.controllers.grasp_highest_controller import highest_point_grasp_pose


problem_position = np.array([0.18679368, 0.24523625, 0.09129775])
problem_position2 = np.array([0.15285242, 0.24341369, 0.09031761])
problem_position3 = np.array([0.14600754, 0.24243534, 0.09408391])

problem_position3[2] += 0.1

tcp_pose_2 = highest_point_grasp_pose(problem_position3)
transform_2 = RigidTransform(tcp_pose_2)

# tcp_pose_2 = np.ascontiguousarray(transform_2.GetAsMatrix4())

add_meshcat_triad(meshcat, "TCP Frame right highest point", X_W_Triad=transform_2)

In [ ]:
desired_grasp_highest_joints_left = np.deg2rad([180, -95, 120, -115, -90, -90])
desired_grasp_highest_joints_left2 = np.deg2rad([0, -85, -120, -65, 90, 90])
desired_grasp_highest_joints_right = np.deg2rad([-180, -85, -120, -65, 90, 90])
desired_grasp_highest_joints_right2 = np.deg2rad([0, -95, 120, -113, -90, -90])

plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

arm_left_index, arm_right_index = arm_indices
plant.SetPositions(plant_context, arm_left_index, desired_grasp_highest_joints_left2)
plant.SetPositions(plant_context, arm_right_index, desired_grasp_highest_joints_right2)
diagram.ForcedPublish(context)

In [ ]:
always_true_fn = lambda *args, **kwargs: True

# Planner without bounds or collisions to check the desirable configuration functionality
planner = DualArmOmplPlanner(
    always_true_fn,
    inverse_kinematics_left_fn,
    inverse_kinematics_right_fn,
)

nudge = np.array([0.0, np.pi / 32, 0.0, 0.0, 0.0, 0.0])  # Problem appears more readily with this nudge
# nudge = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

good_solution_joints_right = np.array([-0.44871921, -0.91104818, 1.39473553, -2.05448368, -1.57079633, 1.12207712])


desirable_goal_configurations_right = [desired_grasp_highest_joints_right, desired_grasp_highest_joints_right2]

path = planner.plan_to_tcp_pose(
    home_joints_left,
    home_joints_right + nudge,
    None,
    tcp_pose_2,
    desirable_goal_configurations_right=desirable_goal_configurations_right,
)
publish_dual_arm_joint_path(path, 2.0, meshcat, diagram, context, *arm_indices)

In [ ]:
np.linalg.norm(desired_grasp_highest_joints_right2 - good_solution_joints_right)

In [ ]:
with np.printoptions(precision=2, suppress=True):
    print(home_joints_right)
    print(desired_grasp_highest_joints_right)
    print(desired_grasp_highest_joints_right2)
    print(good_solution_joints_right)
    print(path[-1][1])

In [ ]:
planner = DualArmOmplPlanner(
    always_true_fn,
    inverse_kinematics_left_fn,
    inverse_kinematics_right_fn,
    joint_bounds_left=joint_bounds,
    joint_bounds_right=joint_bounds,
)

In [ ]:
path = planner.plan_to_tcp_pose(
    home_joints_left,
    home_joints_right + nudge,
    None,
    tcp_pose_2,
)
publish_dual_arm_joint_path(path, 2.0, meshcat, diagram, context, *arm_indices)

In [ ]:
path[-1][1]